# IERG4080 Assignment 1

**Text Classification and Telegram Bot**

Wentao Zhu 1155123308@link.cuhk.edu.hk

[Instructions](https://iems5780.github.io/1819t1/assignments/assignment-1.html)

## Task 1: Text Classification 

### 1. Data Preparation

Notation:
- True = Positive
- False = Negative

In [1]:
!wget !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2018-10-16 14:44:10--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address '!wget'
--2018-10-16 14:44:10--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: 'aclImdb_v1.tar.gz'

aclImdb_v1.tar.gz   100%[===================>]  80.23M  6.49MB/s    in 12s     

2018-10-16 14:44:22 (6.67 MB/s) - 'aclImdb_v1.tar.gz' saved [84125825/84125825]

FINISHED --2018-10-16 14:44:22--
Total wall clock time: 12s
Downloaded: 1 files, 80M in 12s (6.67 MB/s)


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [5]:
import os
import numpy as np
import shutil
import random
import io

In [4]:
os.mkdir('data')

In [5]:
paths = ['aclImdb/test/neg/','aclImdb/train/neg/','aclImdb/test/pos/','aclImdb/train/pos/']
for idx,file_path in enumerate(paths):
    for root, dirs, files in os.walk(file_path):
        print(file_path, len(files))
        for file in files:
            shutil.move(file_path+file, 'data/'+str(idx)+file)

aclImdb/test/neg/ 12500
aclImdb/train/neg/ 12500
aclImdb/test/pos/ 12500
aclImdb/train/pos/ 12500


In [8]:
def isPositive(ss):
    rating = int(ss.split('_')[1].split('.')[0])
#     print(rating)
    if rating>5:
        return True
    else:
        return False

The new dataset is now under data/ 

In [7]:
os.mkdir('data/train')
os.mkdir('data/test')

In [8]:
for root, dirs, files in os.walk('data/', topdown=False):
    num_files = len(files)
    fileset = set(files)
    new_train = set(random.sample(fileset, int(num_files*0.7)))
    new_test = fileset - new_train
    pos_in_train = 0 
    pos_in_test = 0
    for train_file in new_train:
        pos_in_train += isPositive(train_file)
        shutil.move('data/'+train_file, 'data/train/'+train_file)
    for test_file in new_test:
        pos_in_test += isPositive(test_file)
        shutil.move('data/'+test_file, 'data/test/'+test_file)
print('Train Positive: {}/{} = {}'.format(pos_in_train, len(new_train), pos_in_train/len(new_train)))
print('Test Positive: {}/{} = {}'.format(pos_in_test, len(new_test), pos_in_test/len(new_test)))

Train Positive: 17587/35000 = 0.5024857142857143
Test Positive: 7413/15000 = 0.4942


### 2. Using a Naive Bayes Classifier

#### Vectorvization

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [9]:
texts = []
y = [] # Ground Truth
for root, dirs, files in os.walk('data/train'):
    for train_file in files:
        y.append(isPositive(train_file))
        f = io.open('data/train/' + train_file, mode="r", encoding="utf-8")
        texts.append(f.read())
        f.close()

In [210]:
vectorizer = CountVectorizer()
tfvectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(texts)
x_tf = tfvectorizer.fit_transform(texts)

In [11]:
texts_test = []
y_test = [] # Ground Truth
for root, dirs, files in os.walk('data/test'):
    for test_file in files:
        y_test.append(isPositive(test_file))
        f = io.open('data/test/' + test_file, mode="r", encoding="utf-8")
        texts_test.append(f.read())
        f.close()

In [13]:
x_test = vectorizer.transform(texts_test)
x_test_tf = tfvectorizer.transform(texts_test)

#### Train

In [121]:
naiveBayes = MultinomialNB().fit(x,y)

In [122]:
y_predict = naiveBayes.predict(x_test)

In [123]:
print(classification_report(y_test, y_predict))

             precision    recall  f1-score   support

      False       0.83      0.88      0.86      7587
       True       0.87      0.82      0.84      7413

avg / total       0.85      0.85      0.85     15000



In [124]:
accuracy_nb = sum(y_test==y_predict)/len(y_predict)
accuracy_nb

0.8504666666666667

#### Now use TfidfVectorizer

In [125]:
naiveBayes_tf = MultinomialNB().fit(x_tf, y)

In [126]:
y_predict_tf = naiveBayes_tf.predict(x_test_tf)
print(classification_report(y_test, y_predict_tf))
accuracy_nb_tf = sum(y_test==y_predict_tf)/len(y_test)
accuracy_nb_tf

             precision    recall  f1-score   support

      False       0.85      0.88      0.87      7587
       True       0.88      0.85      0.86      7413

avg / total       0.87      0.86      0.86     15000



0.8648666666666667

### 3. Using a Logistic Regression Classifier

In [127]:
from sklearn.linear_model import LogisticRegression
logis = LogisticRegression().fit(x, y)

In [128]:
y_predict_logis = logis.predict(x_test)
print(classification_report(y_test, y_predict_logis))
accuracy_logi = sum(y_test==y_predict_logis)/len(y_test)
accuracy_logi

             precision    recall  f1-score   support

      False       0.89      0.88      0.89      7587
       True       0.88      0.89      0.89      7413

avg / total       0.89      0.89      0.89     15000



0.8872

In [129]:
logis_tf = LogisticRegression().fit(x_tf, y)
y_predict_logis_tf = logis_tf.predict(x_test_tf)
print(classification_report(y_test, y_predict_logis_tf))
accuracy_logi_tf = sum(y_test==y_predict_logis_tf)/len(y_test)
accuracy_logi_tf

             precision    recall  f1-score   support

      False       0.91      0.89      0.90      7587
       True       0.89      0.91      0.90      7413

avg / total       0.90      0.90      0.90     15000



0.8962

### 4. Adding Bi-grams

In [200]:
vectorizerB = CountVectorizer(ngram_range=(1, 2))
tfvectorizerB = TfidfVectorizer(ngram_range=(1, 2))

In [201]:
x = vectorizerB.fit_transform(texts)
x_tf = tfvectorizerB.fit_transform(texts)

In [202]:
x_test = vectorizerB.transform(texts_test)
x_test_tf = tfvectorizerB.transform(texts_test)

#### Naive Bayes with Bi-grams

In [133]:
naiveBayes = MultinomialNB().fit(x,y)
y_predict = naiveBayes.predict(x_test)
print(classification_report(y_test, y_predict))
accuracy_nb = sum(y_test==y_predict)/len(y_predict)
accuracy_nb

             precision    recall  f1-score   support

      False       0.88      0.89      0.88      7587
       True       0.89      0.87      0.88      7413

avg / total       0.88      0.88      0.88     15000



0.8822666666666666

In [134]:
naiveBayes_tf = MultinomialNB().fit(x_tf, y)
y_predict_tf = naiveBayes_tf.predict(x_test_tf)
print(classification_report(y_test, y_predict_tf))
accuracy_nb_tf = sum(y_test==y_predict_tf)/len(y_test)
accuracy_nb_tf

             precision    recall  f1-score   support

      False       0.89      0.90      0.89      7587
       True       0.90      0.88      0.89      7413

avg / total       0.89      0.89      0.89     15000



0.8907333333333334

#### Logistic Regression with Bi-grams

In [203]:
from sklearn.linear_model import LogisticRegression
logis = LogisticRegression().fit(x, y)
y_predict_logis = logis.predict(x_test)
print(classification_report(y_test, y_predict_logis))
accuracy_logi = sum(y_test==y_predict_logis)/len(y_test)
accuracy_logi

             precision    recall  f1-score   support

      False       0.92      0.90      0.91      7587
       True       0.90      0.92      0.91      7413

avg / total       0.91      0.91      0.91     15000



0.9088

In [137]:
logis_tf = LogisticRegression().fit(x_tf, y)
y_predict_logis_tf = logis_tf.predict(x_test_tf)
print(classification_report(y_test, y_predict_logis_tf))
accuracy_logi_tf = sum(y_test==y_predict_logis_tf)/len(y_test)
accuracy_logi_tf

             precision    recall  f1-score   support

      False       0.91      0.88      0.90      7587
       True       0.88      0.91      0.90      7413

avg / total       0.90      0.90      0.90     15000



0.8982

### 5. Using fastText

In [189]:
from fastText import train_supervised

In [190]:
file_for_fr = open('ft_train.txt','w', encoding="utf-8") 
for (idx,item) in enumerate(texts):
    line = ('__label__'+str(y[idx])+' '+item+'\n')
    file_for_fr.write(line)
file_for_fr.close()

file_for_fr = open('ft_test.txt','w', encoding="utf-8") 
for (idx,item) in enumerate(texts_test):
    line = ('__label__'+str(y_test[idx])+' '+item+'\n')
    file_for_fr.write(line)
file_for_fr.close()

In [191]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [192]:
model = train_supervised(input='ft_train.txt')
print_results(*model.test('ft_test.txt'))

N	15000
P@1	0.875
R@1	0.875


In [193]:
model.test('ft_test.txt')

(15000, 0.8745333333333334, 0.8745333333333334)

### 6. Model Persistence

The best model in terms of the accuracy is **Logistic Regression with Bi-grams and CounterVectorizer**

In [212]:
from sklearn.externals import joblib
joblib.dump([vectorizerB,logis], 'model.pkl') # Transformer and Model are both needed for prediction.

['model.pkl']